In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '59 tGravityAcc-energy()-Z',
 '125 tBodyGyro-std()-Y',
 '128 tBodyGyro-mad()-Y',
 '138 tBodyGyro-energy()-Y',
 '165 tBodyGyroJerk-std()-Y',
 '168 tBodyGyroJerk-mad()-Y',
 '178 tBodyGyroJerk-energy()-Y',
 '181 tBodyGyroJerk-iqr()-Y',
 '425 fBodyGyro-mean()-Y',
 '428 fBodyGyro-std()-Y',
 '431 fBodyGyro-mad()-Y',
 '441 fBodyGyro-energy()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '478 fBodyGyro-bandsEnergy()-25,32',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '487 fBodyGyro-bandsEnergy()-1,24',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '7 tBodyAcc-mad()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '204 tBodyAccMag-max()',
 '215 tGravityAccMag-std()',
 '217 tGravityAccMag-max()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '272 fBodyAcc-mad()-X',
 '275 fBodyAcc-max()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '506 fBodyAccMag-max()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']

    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_1 = GAN_data[(GAN_data['Subject'] == 14) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_2 = GAN_data[(GAN_data['Subject'] == 14) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_3 = GAN_data[(GAN_data['Subject'] == 14) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_4 = GAN_data[(GAN_data['Subject'] == 15) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_5 = GAN_data[(GAN_data['Subject'] == 15) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_6 = GAN_data[(GAN_data['Subject'] == 15) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_7 = GAN_data[(GAN_data['Subject'] == 17) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_8 = GAN_data[(GAN_data['Subject'] == 17) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_9 = GAN_data[(GAN_data['Subject'] == 17) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    
    X_train = np.concatenate((X_1, X_2, X_3, X_4, X_5, X_6, X_7, X_8, X_9))
    y_train = [0] * len(X_1) + [1] * len(X_2) + [2] * len(X_3) + [3] * len(X_4) + [4] * len(X_5) + [5] * len(X_6) + [6] * len(X_7) + [7] * len(X_8) + [8] * len(X_9)
    
    return X_train, np.asarray(y_train)

In [4]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 9)
        )
    def forward(self, x):
        return self.network(x)

In [5]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../../saved_models/{model_name}'))

In [6]:
gen = Generator(z_dim = 106)
load_model(gen, "U0A0 Excluded Group 3_gen.param")
gen.eval()
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,0] = 1
    usr_vectors[k,0] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_1 = gen(to_gen).detach().numpy()
y_1 = np.zeros(35)

gen = Generator(z_dim = 106)
load_model(gen, "U0A1 Excluded Group 3_gen.param")
gen.eval()
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,1] = 1
    usr_vectors[k,0] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_2 = gen(to_gen).detach().numpy()
y_2 = np.ones(35)

gen = Generator(z_dim = 106)
load_model(gen, "U0A2 Excluded Group 3_gen.param")
gen.eval()
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,2] = 1
    usr_vectors[k,0] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_3 = gen(to_gen).detach().numpy()
y_3 = np.ones(35) + 1

gen = Generator(z_dim = 106)
load_model(gen, "U1A0 Excluded Group 3_gen.param")
gen.eval()
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,0] = 1
    usr_vectors[k,1] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_4 = gen(to_gen).detach().numpy()
y_4 = np.ones(35) + 2

gen = Generator(z_dim = 106)
load_model(gen, "U1A1 Excluded Group 3_gen.param")
gen.eval()
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,1] = 1
    usr_vectors[k,1] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_5 = gen(to_gen).detach().numpy()
y_5 = np.ones(35) + 3

gen = Generator(z_dim = 106)
load_model(gen, "U1A2 Excluded Group 3_gen.param")
gen.eval()
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,2] = 1
    usr_vectors[k,1] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_6 = gen(to_gen).detach().numpy()
y_6 = np.ones(35) + 4

gen = Generator(z_dim = 106)
load_model(gen, "U2A0 Excluded Group 3_gen.param")
gen.eval()
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,0] = 1
    usr_vectors[k,2] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_7 = gen(to_gen).detach().numpy()
y_7 = np.ones(35) + 5

gen = Generator(z_dim = 106)
load_model(gen, "U2A1 Excluded Group 3_gen.param")
gen.eval()
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,1] = 1
    usr_vectors[k,2] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_8 = gen(to_gen).detach().numpy()
y_8 = np.ones(35) + 6

gen = Generator(z_dim = 106)
load_model(gen, "U2A2 Excluded Group 3_gen.param")
gen.eval()
latent_vectors = get_noise(35, 100)
act_vectors = torch.Tensor(np.zeros((35,3)))
usr_vectors = torch.Tensor(np.zeros((35,3)))
for k in range(35):
    act_vectors[k,2] = 1
    usr_vectors[k,2] = 1
to_gen = torch.cat((latent_vectors, act_vectors, usr_vectors), 1)
fake_features_9 = gen(to_gen).detach().numpy()
y_9 = np.ones(35) + 7

X_test = np.concatenate((fake_features_1, fake_features_2, fake_features_3, fake_features_4, fake_features_5, fake_features_6,
                         fake_features_7, fake_features_8, fake_features_9))
y_test = np.concatenate((y_1, y_2, y_3, y_4, y_5, y_6, y_7, y_8, y_9))

In [7]:
activities = [1, 3, 4]
users = [14, 15, 17]
X_train, y_train = start_data(activities, users, "Activity", sub_features, act_features)

In [8]:
model = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [9]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 4.416366338729858, Final Batch Loss: 2.2094311714172363
Epoch 2, Loss: 4.411351680755615, Final Batch Loss: 2.2092018127441406
Epoch 3, Loss: 4.409950256347656, Final Batch Loss: 2.205338716506958
Epoch 4, Loss: 4.405186176300049, Final Batch Loss: 2.2035961151123047
Epoch 5, Loss: 4.405171155929565, Final Batch Loss: 2.207744598388672
Epoch 6, Loss: 4.406302213668823, Final Batch Loss: 2.201408624649048
Epoch 7, Loss: 4.402022361755371, Final Batch Loss: 2.2073593139648438
Epoch 8, Loss: 4.394659757614136, Final Batch Loss: 2.18928599357605
Epoch 9, Loss: 4.403120994567871, Final Batch Loss: 2.201702833175659
Epoch 10, Loss: 4.396866083145142, Final Batch Loss: 2.203338623046875
Epoch 11, Loss: 4.389291048049927, Final Batch Loss: 2.1896309852600098
Epoch 12, Loss: 4.383824348449707, Final Batch Loss: 2.1918842792510986
Epoch 13, Loss: 4.378310441970825, Final Batch Loss: 2.190980911254883
Epoch 14, Loss: 4.367805004119873, Final Batch Loss: 2.1798012256622314
Epoch 15,

Epoch 116, Loss: 1.7267680764198303, Final Batch Loss: 0.8837472200393677
Epoch 117, Loss: 1.6997599601745605, Final Batch Loss: 0.8845036029815674
Epoch 118, Loss: 1.7007607817649841, Final Batch Loss: 0.8236299157142639
Epoch 119, Loss: 1.7552987337112427, Final Batch Loss: 0.8754104971885681
Epoch 120, Loss: 1.7257468700408936, Final Batch Loss: 0.8550285696983337
Epoch 121, Loss: 1.660182774066925, Final Batch Loss: 0.8348121047019958
Epoch 122, Loss: 1.700490951538086, Final Batch Loss: 0.859285831451416
Epoch 123, Loss: 1.7341635823249817, Final Batch Loss: 0.8709236979484558
Epoch 124, Loss: 1.641904056072235, Final Batch Loss: 0.8366488814353943
Epoch 125, Loss: 1.694413423538208, Final Batch Loss: 0.8769045472145081
Epoch 126, Loss: 1.6317333579063416, Final Batch Loss: 0.8234081864356995
Epoch 127, Loss: 1.6487082839012146, Final Batch Loss: 0.8902210593223572
Epoch 128, Loss: 1.6593859791755676, Final Batch Loss: 0.763741135597229
Epoch 129, Loss: 1.6277579069137573, Final B

Epoch 229, Loss: 1.0802149772644043, Final Batch Loss: 0.5463061928749084
Epoch 230, Loss: 1.1767644882202148, Final Batch Loss: 0.5765054225921631
Epoch 231, Loss: 1.184739887714386, Final Batch Loss: 0.5794706344604492
Epoch 232, Loss: 1.1449797749519348, Final Batch Loss: 0.5859734416007996
Epoch 233, Loss: 1.2442774176597595, Final Batch Loss: 0.6298320889472961
Epoch 234, Loss: 1.1977728605270386, Final Batch Loss: 0.6553043723106384
Epoch 235, Loss: 1.261401116847992, Final Batch Loss: 0.6346442103385925
Epoch 236, Loss: 1.3025575280189514, Final Batch Loss: 0.6316947340965271
Epoch 237, Loss: 1.2060908675193787, Final Batch Loss: 0.5721485614776611
Epoch 238, Loss: 1.1996639966964722, Final Batch Loss: 0.6309945583343506
Epoch 239, Loss: 1.2007994651794434, Final Batch Loss: 0.599519670009613
Epoch 240, Loss: 1.192088782787323, Final Batch Loss: 0.5737956166267395
Epoch 241, Loss: 1.1515591740608215, Final Batch Loss: 0.5313878059387207
Epoch 242, Loss: 1.148260772228241, Final 

Epoch 345, Loss: 0.9472845494747162, Final Batch Loss: 0.4523678719997406
Epoch 346, Loss: 0.9488269686698914, Final Batch Loss: 0.4019750952720642
Epoch 347, Loss: 0.9326653480529785, Final Batch Loss: 0.4767075181007385
Epoch 348, Loss: 0.9024737775325775, Final Batch Loss: 0.45132988691329956
Epoch 349, Loss: 0.9439535737037659, Final Batch Loss: 0.4558892250061035
Epoch 350, Loss: 0.8831825852394104, Final Batch Loss: 0.4448608458042145
Epoch 351, Loss: 0.9624952077865601, Final Batch Loss: 0.4695427417755127
Epoch 352, Loss: 0.9785950779914856, Final Batch Loss: 0.46334338188171387
Epoch 353, Loss: 1.0059814751148224, Final Batch Loss: 0.5302107334136963
Epoch 354, Loss: 0.9335408508777618, Final Batch Loss: 0.48099878430366516
Epoch 355, Loss: 0.9427607953548431, Final Batch Loss: 0.483333945274353
Epoch 356, Loss: 0.9451277852058411, Final Batch Loss: 0.48197221755981445
Epoch 357, Loss: 0.9107107222080231, Final Batch Loss: 0.4175978899002075
Epoch 358, Loss: 0.9195401668548584

Epoch 456, Loss: 0.8712477087974548, Final Batch Loss: 0.3694760203361511
Epoch 457, Loss: 0.8873794078826904, Final Batch Loss: 0.4510103762149811
Epoch 458, Loss: 0.8729475736618042, Final Batch Loss: 0.47833919525146484
Epoch 459, Loss: 0.8523648083209991, Final Batch Loss: 0.4594773054122925
Epoch 460, Loss: 0.8919640779495239, Final Batch Loss: 0.40815186500549316
Epoch 461, Loss: 0.8211559355258942, Final Batch Loss: 0.4459083080291748
Epoch 462, Loss: 0.8648236393928528, Final Batch Loss: 0.4072718918323517
Epoch 463, Loss: 0.9499256014823914, Final Batch Loss: 0.5183457136154175
Epoch 464, Loss: 0.8210845291614532, Final Batch Loss: 0.42006590962409973
Epoch 465, Loss: 0.8412117660045624, Final Batch Loss: 0.3903234302997589
Epoch 466, Loss: 0.8532694876194, Final Batch Loss: 0.37877193093299866
Epoch 467, Loss: 0.8174645304679871, Final Batch Loss: 0.42556747794151306
Epoch 468, Loss: 0.8040016293525696, Final Batch Loss: 0.4000927805900574
Epoch 469, Loss: 0.8697860538959503,

...

Epoch 4529, Loss: 0.13483154401183128, Final Batch Loss: 0.07579442858695984
Epoch 4530, Loss: 0.12805141881108284, Final Batch Loss: 0.08050937205553055
Epoch 4531, Loss: 0.14644381776452065, Final Batch Loss: 0.08424212038516998
Epoch 4532, Loss: 0.13631830364465714, Final Batch Loss: 0.08257730305194855
Epoch 4533, Loss: 0.11885594949126244, Final Batch Loss: 0.06859140843153
Epoch 4534, Loss: 0.13387836515903473, Final Batch Loss: 0.08387482166290283
Epoch 4535, Loss: 0.10020343214273453, Final Batch Loss: 0.05672154575586319
Epoch 4536, Loss: 0.18393444269895554, Final Batch Loss: 0.09243210405111313
Epoch 4537, Loss: 0.11783010885119438, Final Batch Loss: 0.0481075681746006
Epoch 4538, Loss: 0.16371850669384003, Final Batch Loss: 0.05665694177150726
Epoch 4539, Loss: 0.18955054879188538, Final Batch Loss: 0.0675576776266098
Epoch 4540, Loss: 0.09941750764846802, Final Batch Loss: 0.04616176709532738
Epoch 4541, Loss: 0.13583776354789734, Final Batch Loss: 0.08152443170547485
Epoc

Epoch 4644, Loss: 0.09400278329849243, Final Batch Loss: 0.04936987906694412
Epoch 4645, Loss: 0.13532926887273788, Final Batch Loss: 0.06064945459365845
Epoch 4646, Loss: 0.1240498535335064, Final Batch Loss: 0.0630328431725502
Epoch 4647, Loss: 0.1348361000418663, Final Batch Loss: 0.0432954803109169
Epoch 4648, Loss: 0.16613999754190445, Final Batch Loss: 0.08598588407039642
Epoch 4649, Loss: 0.09510697796940804, Final Batch Loss: 0.05455735698342323
Epoch 4650, Loss: 0.10416391864418983, Final Batch Loss: 0.038030724972486496
Epoch 4651, Loss: 0.13451865315437317, Final Batch Loss: 0.06042011082172394
Epoch 4652, Loss: 0.11054152995347977, Final Batch Loss: 0.043644070625305176
Epoch 4653, Loss: 0.10965598747134209, Final Batch Loss: 0.056436218321323395
Epoch 4654, Loss: 0.09706661105155945, Final Batch Loss: 0.059950657188892365
Epoch 4655, Loss: 0.10431493446230888, Final Batch Loss: 0.050308555364608765
Epoch 4656, Loss: 0.17980553954839706, Final Batch Loss: 0.1035552993416786

Epoch 4755, Loss: 0.10263919830322266, Final Batch Loss: 0.05932891368865967
Epoch 4756, Loss: 0.13700227066874504, Final Batch Loss: 0.07648409157991409
Epoch 4757, Loss: 0.0776349138468504, Final Batch Loss: 0.05451839044690132
Epoch 4758, Loss: 0.14625797793269157, Final Batch Loss: 0.04895472154021263
Epoch 4759, Loss: 0.09771542623639107, Final Batch Loss: 0.055487100034952164
Epoch 4760, Loss: 0.10458711162209511, Final Batch Loss: 0.05832797661423683
Epoch 4761, Loss: 0.1455635391175747, Final Batch Loss: 0.05639495328068733
Epoch 4762, Loss: 0.11982619017362595, Final Batch Loss: 0.08385822176933289
Epoch 4763, Loss: 0.13226095587015152, Final Batch Loss: 0.06649895757436752
Epoch 4764, Loss: 0.10910915210843086, Final Batch Loss: 0.05383291840553284
Epoch 4765, Loss: 0.1168702095746994, Final Batch Loss: 0.050903476774692535
Epoch 4766, Loss: 0.14261089637875557, Final Batch Loss: 0.08013202250003815
Epoch 4767, Loss: 0.1574331745505333, Final Batch Loss: 0.10883758217096329
E

Epoch 4871, Loss: 0.16072913259267807, Final Batch Loss: 0.07457903027534485
Epoch 4872, Loss: 0.11051236093044281, Final Batch Loss: 0.03611409664154053
Epoch 4873, Loss: 0.07866564020514488, Final Batch Loss: 0.03608334809541702
Epoch 4874, Loss: 0.1271159127354622, Final Batch Loss: 0.08436872810125351
Epoch 4875, Loss: 0.1145530454814434, Final Batch Loss: 0.06910238415002823
Epoch 4876, Loss: 0.19749878346920013, Final Batch Loss: 0.061747729778289795
Epoch 4877, Loss: 0.1164025217294693, Final Batch Loss: 0.05958864465355873
Epoch 4878, Loss: 0.14302335679531097, Final Batch Loss: 0.054524146020412445
Epoch 4879, Loss: 0.15578358620405197, Final Batch Loss: 0.09798116981983185
Epoch 4880, Loss: 0.1275239996612072, Final Batch Loss: 0.08095454424619675
Epoch 4881, Loss: 0.13412423431873322, Final Batch Loss: 0.04992229491472244
Epoch 4882, Loss: 0.14895852655172348, Final Batch Loss: 0.04605540633201599
Epoch 4883, Loss: 0.12574884667992592, Final Batch Loss: 0.08635839819908142
E

Epoch 4982, Loss: 0.09860795736312866, Final Batch Loss: 0.05375821143388748
Epoch 4983, Loss: 0.181132934987545, Final Batch Loss: 0.0914762020111084
Epoch 4984, Loss: 0.11567419022321701, Final Batch Loss: 0.05343172699213028
Epoch 4985, Loss: 0.0884178951382637, Final Batch Loss: 0.029726438224315643
Epoch 4986, Loss: 0.10470554418861866, Final Batch Loss: 0.02894890494644642
Epoch 4987, Loss: 0.17364854738116264, Final Batch Loss: 0.05629913881421089
Epoch 4988, Loss: 0.14501137286424637, Final Batch Loss: 0.058561891317367554
Epoch 4989, Loss: 0.13223803043365479, Final Batch Loss: 0.03754442185163498
Epoch 4990, Loss: 0.1268911175429821, Final Batch Loss: 0.06956856697797775
Epoch 4991, Loss: 0.20192745327949524, Final Batch Loss: 0.12435861676931381
Epoch 4992, Loss: 0.09834598377346992, Final Batch Loss: 0.047008465975522995
Epoch 4993, Loss: 0.09896162152290344, Final Batch Loss: 0.03966379165649414
Epoch 4994, Loss: 0.10618475452065468, Final Batch Loss: 0.05082065612077713
E

In [22]:
softmax = nn.Softmax(dim = 1)
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[ 0  0  0 35  0  0  0  0  0]
 [ 0 34  0  0  0  0  1  0  0]
 [ 5  0 29  0  0  0  0  0  1]
 [ 0  0  0 22  0  0  6  7  0]
 [ 0  0  0  0 35  0  0  0  0]
 [ 0  0  3  0  0 32  0  0  0]
 [ 0  0  0  0  0  0 35  0  0]
 [ 0  7  0  0  0  0  3 25  0]
 [ 0  0  3  0  0  3  0  0 29]]
              precision    recall  f1-score   support

         0.0    0.00000   0.00000   0.00000        35
         1.0    0.82927   0.97143   0.89474        35
         2.0    0.82857   0.82857   0.82857        35
         3.0    0.38596   0.62857   0.47826        35
         4.0    1.00000   1.00000   1.00000        35
         5.0    0.91429   0.91429   0.91429        35
         6.0    0.77778   1.00000   0.87500        35
         7.0    0.78125   0.71429   0.74627        35
         8.0    0.96667   0.82857   0.89231        35

    accuracy                        0.76508       315
   macro avg    0.72042   0.76508   0.73660       315
weighted avg    0.72042   0.76508   0.73660       315



In [24]:
# GET RID OF 2, 5, 8 for JUST DYNAMIC
(0+0.89474+0.47826+1+0.91429+0.875+0.74627)/6

0.8180933333333335